In [ ]:
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15
!pip install transformers
!pip install contractions
!pip install emoji==1.6.3
!pip install emot==2.1

In [ ]:
import numpy as np
import pandas as pd

import torch
import transformers as ppb
import tensorflow as tf
import tensorflow_hub as hub

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

In [ ]:
from google.colab import auth
auth.authenticate_user()
!curl https://sdk.cloud.google.com | bash


!gcloud init

In [ ]:
#!gsutil -m cp gs://masterthesisbert/bert_model_BERT_RE_long/bert-reddit-vocab.txt gs://masterthesisbert/bert_model_BERT_RE_wide/bert-reddit-vocab.txt

In [ ]:
my_bert = False
my_first_bert=False
my_second_bert=False


if (my_bert == True):

    #Important. Set your parameters
    bert_dir_name = "masterthesisbert"
    !mkdir {bert_dir_name}
    path_to_bert_folder = "masterthesisbert/bert_model_BERT_RE_long"
    path_last_bert_folder = "bert_model_BERT_RE_long"

    #!gsutil -m cp -r gs://{path_to_bert_folder} /content/{bert_dir_name}

    #some parameters should be put here as well

    !gsutil cp gs://masterthesisbert/{path_last_bert_folder}/bert-reddit-vocab.txt /content/bert-vocab.txt

    !gsutil cp gs://masterthesisbert/{path_last_bert_folder}/model.ckpt-1000000.data-00000-of-00001 /content/
    !gsutil cp gs://masterthesisbert/{path_last_bert_folder}/bert_config.json /content/
    !gsutil cp gs://masterthesisbert/{path_last_bert_folder}/model.ckpt-1000000.index /content/
    !gsutil cp gs://masterthesisbert/{path_last_bert_folder}/model.ckpt-1000000.meta /content/
    
    !transformers-cli convert --model_type bert --tf_checkpoint /content/model.ckpt-1000000 --config /content/bert_config.json --pytorch_dump_output /content/pytorch_model.bin

    # importing bert-base, tokenizers, and models from libraries
    model_class, tokenizer_class, pretrained_weights, pretrained_tokenizer = (ppb.BertModel, ppb.BertTokenizer, '/content/pytorch_model.bin', '/content/bert-vocab.txt')
    tokenizer = tokenizer_class.from_pretrained(pretrained_tokenizer)
    model = model_class.from_pretrained(pretrained_weights, config="bert_config.json")

else:

    # importing bert-base, tokenizers, and models from libraries
    model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)

In [ ]:
aladag = False
reddit_500 = False
eight_classes=False
depression=False
labaled_data = False
preprocessing_done = True
rsdd=True
preprocessing_done_final=True
balance_smhd = False 

In [ ]:
if balance_smhd:
    mental_condition_name = "SMHD_train.csv"
    mental_condition_name_test = "SMHD_test.csv"
    mental_condition_name_valid = "SMHD_dev.csv"

    !gsutil cp  gs://masterthesisbert/{mental_condition_name} /content/{mental_condition_name}
    !gsutil cp  gs://masterthesisbert/{mental_condition_name_test} /content/{mental_condition_name_test}
    !gsutil cp  gs://masterthesisbert/{mental_condition_name_valid} /content/{mental_condition_name_valid}

    df_mental = pd.read_csv(mental_condition_name)
    df_mental_valid = pd.read_csv(mental_condition_name_test)
    df_mental_test = pd.read_csv(mental_condition_name_valid)
    df_mental = pd.concat([df_mental, df_mental_valid], axis=0).reset_index()

    df_mental_control2 = df_mental[df_mental['label'] == 'control'].copy().sample(frac=.98)
    df_mental_test_control2 = df_mental_test[df_mental_test['label'] == 'control'].copy().sample(frac=.98)
    df_mental_test.drop(df_mental_test_control2.index, inplace=True)
    df_mental_test.reset_index(inplace=True)
    df_mental.drop(df_mental_control2.index, inplace=True)
    df_mental.reset_index(inplace=True)
    a = ["control", "adhd", "anxiety", "depression","bipolar", "autism", "ptsd","ocd", "eating", "schizophrenia"]
    df_mental = df_mental[df_mental['label'].isin(a)]
    df_mental_test = df_mental_test[df_mental_test['label'].isin(a)]
    b = ["[removed]", ".", "Added", "No", "Nice"]
    df_mental = df_mental[~df_mental['text'].isin(b)]
    df_mental_test = df_mental_test[~df_mental_test['text'].isin(b)]
    df_mental = df_mental[~df_mental['text'].isna()]
    df_mental_test = df_mental_test[~df_mental_test['text'].isna()]

    import re
    import contractions 
    from bs4 import BeautifulSoup

    def clean_text_small(text):

        text = re.sub(r'\n', '', text)
        ## Remove html content
        text = BeautifulSoup(text, "html.parser").text

        ## Remove contractions
        text = contractions.fix(text)
        text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        text = text.lower()
        return text

    df_mental["text"] = df_mental.apply(lambda x: clean_text_small(x["text"]), axis=1)
    df_mental_test["text"] = df_mental_test.apply(lambda x: clean_text_small(x["text"]), axis=1)
    df_mental_test

    df_mental["text"].dropna(axis =0, inplace=True)
    df_mental_test["text"].dropna(axis =0, inplace=True)

    df_mental = df_mental[df_mental['text'].str.split().str.len().gt(10)]

    df_mental_test = df_mental_test[df_mental_test['text'].str.split().str.len().gt(10)]

    sample_amounts = {'adhd': 1500,
                      'anxiety': 1500,
                      'autism': 900,
                      'bipolar': 1000,
                      'control': 1500,
                      'depression': 1500,
                      'eating': 350,
                      'ocd': 500,
                      'ptsd': 800,
                      'schizophrenia': 450}


    df_mental = (
        df_mental.groupby('label').apply(lambda g: g.sample(
            # lookup number of samples to take
            n=sample_amounts[g.name],
            # enable replacement if len is less than number of samples expected
            replace=len(g) < sample_amounts[g.name]  
        )).droplevel(0)
    )

    n=2000
    df_mental_test = df_mental_valid.sample(n, replace=False)
    df_mental.drop(columns=["level_0"], inplace = True)
    df_mental = df_mental.sample(frac=1).reset_index(drop=True)
    df_mental_test = df_mental_valid.sample(frac=1).reset_index(drop=True)
    df_mental.to_csv("df_mental_balanced_preprocessed.csv")
    df_mental_test.to_csv("df_mental_valid_preprocessed.csv")

    # Is commented out so that I dont overwrite my code 
    #!gsutil cp  /content/df_mental_balanced_preprocessed.csv gs://masterthesisbert/df_mental_balanced_preprocessed.csv
    #!gsutil cp  /content/df_mental_valid_preprocessed.csv gs://masterthesisbert/df_mental_valid_preprocessed.csv

In [ ]:
if preprocessing_done:
    !gsutil cp gs://masterthesisbert/df_mental_balanced_preprocessed.csv /content/df_mental_balanced_preprocessed.csv
    !gsutil cp gs://masterthesisbert/df_mental_valid_preprocessed.csv /content/df_mental_valid_preprocessed.csv
    !gsutil cp gs://masterthesisbert/df_mental_balanced_preprocessed_preprocessed.csv /content/df_mental_balanced_preprocessed_preprocessed.csv
    !gsutil cp gs://masterthesisbert/df_mental_valid_preprocessed_preprocessed.csv /content/df_mental_valid_preprocessed_preprocessed.csv
    !gsutil cp gs://masterthesisbert/rsdd_train_preprocessed.csv /content/rsdd_train_preprocessed.csv
    !gsutil cp gs://masterthesisbert/rsdd_valid_preprocessed.csv /content/rsdd_valid_preprocessed.csv
    !gsutil cp gs://masterthesisbert/reddit_500_final_train.csv /content/reddit_500_final_train.csv 
    !gsutil cp gs://masterthesisbert/reddit_500_final_val.csv /content/reddit_500_final_val.csv
    !gsutil cp gs://masterthesisbert/Aladag_sample_preprocessed.csv /content/Aladag_sample_preprocessed.csv
    !gsutil cp gs://masterthesisbert/Aladag_labeled_preprocessed.csv /content/Aladag_labeled_preprocessed.csv
    


else:


    if aladag:
        !gsutil cp gs://masterthesisbert/Aladag_sample.csv /content/Aladag_sample.csv
        !gsutil cp gs://masterthesisbert/SuicideAladag2018_preprocessed.csv /content

        

    elif eight_classes==True:
        !gsutil cp gs://masterthesisbert/df_mental_balanced.csv  /content/df_mental_balanced.csv
        !gsutil cp gs://masterthesisbert/df_mental_valid_preprocessed.csv /content/df_mental_valid_preprocessed.csv

    elif reddit_500:
        !gsutil cp gs://masterthesisbert/500_Reddit_users_posts_labels_preprocessed.csv /content/500_Reddit_users_posts_labels_preprocessed.csv

    elif rsdd: 
        !gsutil cp gs://masterthesisbert/RSDD_train.csv /content/RSDD_train.csv
        !gsutil cp gs://masterthesisbert/RSDD_valid.csv /content/RSDD_valid.csv


In [ ]:
# Look at the below comments to determine whether you want to output the 2-dimensional or 3-dimensional BERT features.
def getFeatures(batch_1, tokenizer, model):

  tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, padding='max_length', truncation=True, max_length=256)))

  max_len = 0
  for i in tokenized.values:
      if len(i) > max_len:
          max_len = len(i)

  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])


  attention_mask = np.where(padded != 0, 1, 0)
  attention_mask.shape


  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)
  
  with torch.no_grad():
      last_hidden_states = model(input_ids, attention_mask=attention_mask)
  """
  hidden_states = last_hidden_states[2]

  check_1 = torch.add(hidden_states[12], hidden_states[11])
  check_2 = torch.add(hidden_states[10], hidden_states[9])
  check_3 = torch.add(check_1, check_2)

  #features = check_3.numpy()
  features = check_3[:,0,:].numpy()
  """
  
  features_2D = last_hidden_states[0][:,0,:].numpy() # use this line if you want the 2D BERT features
  features_3D = last_hidden_states[0].numpy() # use this line if you want the 3D BERT features 

  return features_2D, features_3D 

In [ ]:
bucket_name = 'masterthesisbert'

In [ ]:
#when using google drive
drive = False
if drive==True:
    addition = "drive/MyDrive/"
else:
    addition = ""

In [ ]:
import math

if(preprocessing_done==False):
    if aladag == True:
      
        df_name = "SuicideAladag2018_preprocessed.csv"

        
    elif reddit_500 == True:

        df_name = "500_Reddit_users_posts_labels_preprocessed.csv"

    elif eight_classes:

        if labaled_data:

            df_name = "df_mental_valid_preprocessed.csv"

        else: 

            df_name = "df_mental_balanced_preprocessed.csv"

    elif rsdd:

        if labaled_data:

            df_name = "RSDD_train.csv"

        else: 

            df_name = "RSDD_valid.csv"

    df = pd.read_csv(df_name)

In [ ]:
if eight_classes and preprocessing_done==False:
    output = pd.read_csv(df_name)

In [ ]:
if (preprocessing_done==False and aladag==True):
    # co zrobic z tym koda 
    df_annotated = output[~np.isnan(output["binary_annotation"])].copy()
    df_annotated = df_annotated[(df_annotated["subreddit"] == 'SuicideWatch') | (df_annotated["subreddit"] == 'Showerthoughts')]
    df_annotated.to_csv("labeled_Aladag.csv")
    df_annotated.subreddit.value_counts()
    !gsutil cp /content/labeled_Aladag.csv gs://{bucket_name}/labeled_Aladag.csv
    print(len(df))
    df = df[~df["Unnamed: 0"].isin(df_annotated["Unnamed: 0"])].copy()
    #We have to perform now everything for two datasets 

    print(len(df))

    df = df[(df.subreddit == "SuicideWatch") | (df.subreddit == "Showerthoughts")].copy()
    print(len(df))

    samples_per_group_dict = {'SuicideWatch': 5000, 
                              'Showerthoughts': 5000}
                              
    output = df.groupby('subreddit').apply(lambda group: group.sample(samples_per_group_dict[group.name])).reset_index(drop = True)

    output = output.sample(frac = 1)
    output.subreddit.value_counts()
    # for training purposes we have to assume that all post from SuicideWatch in df are 1 and all posts from Showerthoughts are 0 
    output["binary_annotation"] = output.apply(lambda x: 1 if (x["subreddit"] == "SuicideWatch") else 0, axis=1)
    output.reset_index(inplace=True)
    output.to_csv("Aladag_sample.csv")
    output = pd.read_csv("Aladag_sample.csv")
    output

#Clean Text Aladag so that we can create embeddings on cleaned text

In [ ]:
bucket_name="masterthesisbert"

In [ ]:
!gsutil cp gs://{bucket_name}/Emoji_Dict.p /content/Emoji_Dict.p 
!gsutil cp gs://{bucket_name}/Emoticon_Dict.p /content/Emoticon_Dict.p 

Copying gs://masterthesisbert/Emoji_Dict.p...
/ [1 files][ 88.1 KiB/ 88.1 KiB]                                                
Operation completed over 1 objects/88.1 KiB.                                     
Copying gs://masterthesisbert/Emoticon_Dict.p...
/ [1 files][  3.4 KiB/  3.4 KiB]                                                
Operation completed over 1 objects/3.4 KiB.                                      


In [ ]:
from emot.emo_unicode import EMOTICONS
import pickle
import emoji
with open('Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)

with open('Emoticon_Dict.p', 'rb') as fp:
    Emoticon_Dict = pickle.load(fp)

Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

In [ ]:
import re
import contractions
from bs4 import BeautifulSoup

def convert_emoticons_to_word(text):
    for emot in EMOTICONS:

        text = re.sub(u'(' + emot + ')', "_".join(EMOTICONS[emot].replace(",", "").split()), text)
    return text


def convert_emojis_to_word(text):

    text = emoji.demojize(text, delimiters=("", ""))

    return text

def clean_text(text):

    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)

    text = re.sub(r"\-", " ", text)

    text = re.sub(r"\=", " ", text)
    text = re.sub(r"\,", " , ", text)
    text = re.sub(r"\.", " . ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\<", " ", text)
    text = re.sub(r"\;", " ; ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r",", " , ", text)


    # text = re.sub(r"\'s", " ", text)

    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r" ve ", " have ", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " d ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"y'", "you ", text)

    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " usa ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", " 911 ", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)

    #text = re.sub(r"[^A-Za-z0-9^,!.]", " ", text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)

    ## Remove html content
    text = BeautifulSoup(text, "html.parser").text

    ## Remove contractions
    text = contractions.fix(text)

    return text

#Labeled data

In [ ]:
labeled_data=False

In [ ]:
if(preprocessing_done==False):
    if labeled_data and aladag:

      output = df_annotated.copy()
      
    else:
      output = df.copy()

if eight_classes:
    column = "text"
elif rsdd:
    column = "posts"


else: column = "selftext"

In [ ]:
def preprocess_text(output):
    output[column] = output.apply(lambda x: convert_emojis_to_word(str(x[column])), axis=1)

    output[column] = output.apply(lambda x: convert_emoticons_to_word(x[column]), axis=1)

    output[column] = output.apply(lambda x: re.sub(r'^https?:\/\/.*[\r\n]*', '', x[column], flags=re.MULTILINE), axis=1)

    output[column] = output.apply(lambda x: x[column].lower(), axis=1)

    output[column] = output.apply(lambda x: clean_text(x[column]), axis=1)

    return output

In [ ]:
# supportive is not ideation, everything else is

if (reddit_500==True and preprocessing_done == False):

    df["Label"] = df.apply(lambda x: 1 if x["Label"] in ["Ideation", "Attempt", "Behavior"] else 0, axis=1)
    
    df = preprocess_text(df)

    # for test we do it like that 
    df1 = df.query('Label == 0').sample(3250)
    df2 = df.query('Label == 1').sample(3250)
    df = pd.concat([df1, df2])
    dftest1 = df.query('Label == 0').sample(1000)
    dftest2 = df.query('Label == 1').sample(1000)
    dftest = pd.concat([dftest1, dftest2])
    df = pd.merge(df,dftest, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
    dftest.reset_index(inplace=True)
    df.reset_index(inplace=True)
    df.to_csv("reddit_500_final_train.csv")
    dftest.to_csv("reddit_500_final_val.csv")
    #if we wanna load it to a disc
    #!gsutil cp /content/reddit_500_final_train.csv gs://masterthesisbert/reddit_500_final_train.csv
    #!gsutil cp /content/reddit_500_final_val.csv gs://masterthesisbert/reddit_500_final_val.csv


elif(rsdd == True and preprocessing_done ==False):

    output = output[output['posts'].str.split().str.len().gt(10)]

    output = preprocess_text(output)

    output["label"] = output.apply(lambda x: 1 if x["label"] in ["depression"] else 0, axis=1)


    if labaled_data: 
        df1 = output.query('label == 0').sample(1000)
        df2 = output.query('label == 1').sample(1000)
        output = pd.concat([df1, df2])

        output = output.sample(frac=1)

        output.to_csv("rsdd_valid_preprocessed.csv")

        !gsutil cp /content/rsdd_valid_preprocessed.csv gs://masterthesisbert/rsdd_valid_preprocessed.csv


    else:

        df1 = output.query('label == 0').sample(2500)
        df2 = output.query('label == 1').sample(2500)
        output = pd.concat([df1, df2])

        output = output.sample(frac=1)

        output.to_csv("rsdd_train_preprocessed.csv")

        !gsutil cp /content/rsdd_train_preprocessed.csv gs://masterthesisbert/rsdd_train_preprocessed.csv

elif(aladag == True and preprocessing_done == False):
    output = preprocess_text(output)

    if labaled_data:

        output.to_csv("Aladag_labeled_preprocessed.csv")
        #!gsutil cp /content/Aladag_labeled_preprocessed.csv gs://{bucket_name}/Aladag_labeled_preprocessed.csv

    else:

        output.to_csv("Aladag_sample_preprocessed.csv")
        #!gsutil cp /content/Aladag_sample_preprocessed.csv gs://{bucket_name}/Aladag_sample_preprocessed.csv



"""
elif(eight_classes==True and preprocessing_done == False):


    output = output[output['posts'].str.split().str.len().gt(10)]


    output = preprocess_text(output)


    if labaled_data:
        output.to_csv("df_mental_valid_preprocessed.csv")
        #Its commented out so that I accidentaly dont overwrite my files 
        #!gsutil cp /content/df_mental_valid_preprocessed.csv gs://{bucket_name}/df_mental_valid_preprocessed.csv

    else:
        output.to_csv("df_mental_balanced_preprocessed.csv")
        #Its commented out so that I accidentaly dont overwrite my files 

        #!gsutil cp /content/df_mental_balanced_preprocessed.csv gs://{bucket_name}/df_mental_balanced_preprocessed.csv

    
"""

In [ ]:
labaled_data=True

In [ ]:
if preprocessing_done:
  
    if labaled_data:
        if aladag:
            output = pd.read_csv("Aladag_labeled_preprocessed.csv")

        elif eight_classes and preprocessing_done_final==False:

            output = pd.read_csv("df_mental_valid_preprocessed.csv")

            output = output.astype({"text": str, "label":str}, errors='raise')
            output = output[output.label != "nan"]
            output.reset_index(drop=True, inplace=True)
            column = "text"
            output = preprocess_text(output)


        elif reddit_500:


            dftest = pd.read_csv("reddit_500_final_val.csv")

        elif rsdd:

            output = pd.read_csv("rsdd_valid_preprocessed.csv")


    else:
        if aladag:

            output = pd.read_csv("Aladag_sample_preprocessed.csv")

        elif eight_classes and preprocessing_done_final==False:

            output = pd.read_csv("df_mental_balanced_preprocessed.csv")

            output = output.astype({"text": str, "label":str}, errors='raise')
            output = output[output.label != "nan"]
            output.reset_index(drop=True, inplace=True)
            column = "text"
            output = preprocess_text(output)

        elif reddit_500:


            df = pd.read_csv("reddit_500_final_train.csv")

        elif rsdd:

            output = pd.read_csv("rsdd_train_preprocessed.csv")

In [ ]:
preprocessing_done_final=True
labaled_data = True

In [ ]:
if preprocessing_done_final:
    if eight_classes:

        if labaled_data:
        
            output = pd.read_csv("df_mental_valid_preprocessed_preprocessed.csv")

        else:

            output = pd.read_csv("df_mental_balanced_preprocessed_preprocessed.csv")

else:
    if preprocessing_done: 


        if labaled_data:
            if eight_classes:
              output.to_csv("df_mental_valid_preprocessed_preprocessed.csv")
              #Its commented out just not to accidentally overwrite              
              #!gsutil cp  /content/df_mental_valid_preprocessed_preprocessed.csv gs://masterthesisbert/df_mental_valid_preprocessed_preprocessed.csv 
        else: 
            if eight_classes:
              output.to_csv("df_mental_balanced_preprocessed_preprocessed.csv")
              #Its commented out just not to accidentally overwrite              


              #!gsutil cp  /content/df_mental_balanced_preprocessed_preprocessed.csv gs://masterthesisbert/df_mental_balanced_preprocessed_preprocessed.csv 

In [ ]:
if aladag:
    output = output.rename(columns={'selftext': 0, 'binary_annotation': 1})

elif eight_classes:

    output = output.rename(columns={'text': 0, 'label': 1})
    final_table_columns = [0, 1]
    output = output[output.columns.intersection(final_table_columns)]
    output=output.dropna(axis=0)
    len(output)

elif reddit_500:

    df = df.rename(columns={'selftext': 0, 'Label': 1})
    dftest = dftest.rename(columns={'selftext': 0, 'Label': 1})
    final_table_columns = [0, 1]
    df = df[df.columns.intersection(final_table_columns)]
    dftest = dftest[dftest.columns.intersection(final_table_columns)]
    df=df.dropna(axis=0)
    dftest=dftest.dropna(axis=0)

elif rsdd:

    output = output.rename(columns={'posts': 0, 'label': 1})
    final_table_columns = [0, 1]
    output = output[output.columns.intersection(final_table_columns)]
    output=output.dropna(axis=0)

In [ ]:
if reddit_500:
  df_train_x = df.copy()
  df_test_x = dftest.copy()

else:
  df_train_x = output.copy()

In [ ]:
output

In [ ]:
def text_to_embeddings(df_train_x, tokenizer, model, start_from = 0):
    
    max_features = 256
    big_step = 1000
    if start_from == 0:
        a = 0
    else: 
        a = start_from/big_step
    b = len(df_train_x)
    for j in list(range(start_from,b,big_step)):
        print(j, "j")
        if (b - j) < big_step:
            # for the last step
            start = j
            stop = b
            df_short = df_train_x[start:stop].copy()
            print("last shot")
            bert_features_first_2D, bert_features_first_3D  = getFeatures(df_short, tokenizer, model)
        else:
            for i in list(range(j,j + big_step,100)):
                print(i, "i")
                if ((i == 0) or (i%big_step==0)):
                    print("starting")
                    start = i
                    stop = i + 100
                    df_short = df_train_x[start:stop].copy()
                    bert_features_first_2D, bert_features_first_3D = getFeatures(df_short, tokenizer, model)                
                elif i == list(range(j,j+big_step+1,100))[-1]:
                    #do nothing
                    print("we did nothing")
                    '''    
                    elif i == list(range(j,j+big_step,100))[-1]:
                    start = i
                    stop = len(df_train_x)
                    df_short = df_train_x[start:stop].copy()
                    bert_features_third= getFeatures(df_short, tokenizer, model)
                    bert_features_third = np.concatenate((bert_features_third, bert_features_first), axis=0)
                    '''
                else:
                    print("next step")
                    start = i
                    stop = i + 100
                    df_short_2 = df_train_x[start:stop].copy()
                    bert_features_second_2D, bert_features_second_3D =  getFeatures(df_short_2, tokenizer, model)

                    bert_features_first_2D = np.concatenate((bert_features_first_2D, bert_features_second_2D), axis=0)
         
                    bert_features_first_3D = np.concatenate((bert_features_first_3D, bert_features_second_3D), axis=0)

        name_3D = f"MyBERT_wide_full-training-features_3D_labeled_rsdd_val{a}.txt"

        name_2D = f"MyBERT_wide_full-training-features_2D_labeled_rsdd_val{a}.txt"
        
        # reshaping the array from 3D
        # matrice to 2D matrice.
        bert_features_first_3D = bert_features_first_3D.reshape(bert_features_first_3D.shape[0], -1)
  
        # saving reshaped array to file.
        
        np.savetxt(name_2D, bert_features_first_2D)
        np.savetxt(name_3D, bert_features_first_3D)


        !gsutil cp /content/{name_2D} gs://masterthesisbert/embeddings_labeled/PsychBERT_long/
        !gsutil cp /content/{name_3D} gs://masterthesisbert/embeddings_labeled/PsychBERT_long/


        a+=1 
    return bert_features_first_2D, bert_features_first_3D


In [ ]:
len(df_train_x)

10000

In [ ]:
#for reddit_500 change df_train_x to df_val_x 

bert_features_train_2D, bert_features_train_3D = text_to_embeddings(df_train_x, tokenizer, model)
